In [ ]:
!pip install keras-nlp
!pip install pandas
!pip install -q --upgrade keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.


In [ ]:
import os
import time
import keras
import keras_nlp
import pandas as pd
import tensorflow as tf
import os
from google.colab import drive
from openpyxl import Workbook
from openpyxl import load_workbook

os.environ["KERAS_BACKEND"] = "tensorflow"

# Mount Google Drive
drive.mount('/content/drive')
keras.mixed_precision.set_global_policy("mixed_float16")

Mounted at /content/drive


# GPT2 - Large

In [ ]:
preprocessor_large = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_large_en",
)
gpt2_large = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_large_en", preprocessor=preprocessor_large
)


In [ ]:
wb = Workbook()
wb = load_workbook(filename='/content/drive/MyDrive/stat.xlsx')
sheet = wb['Feuil1']

# GPT 2 - Large
i=2
for cell in sheet['A'][1:]:
    print(i-1, cell.value)
    sheet['B' + str(i) ] = gpt2_large.generate(cell.value, max_length=280)
    wb.save('/content/drive/MyDrive/stat.xlsx')
    i+=1
    if i > 31:
      break
wb.close()


1 What are the common causes of headaches?
2 How do over-the-counter pain relievers like ibuprofen work?
3 Can you explain the symptoms of a migraine?
4 What are the potential side effects of long-term medication use?
5 What are the differences between viral and bacterial infections?
6 Can you describe the stages of wound healing?
7 What is the significance of a healthy diet in preventing illness?
8 How does exercise affect mental health?
9 What are the warning signs of a heart attack?
10 How do vaccines work to prevent disease?
11 What is the role of antibiotics in treating bacterial infections?
12 Can you explain the concept of antibiotic resistance?
13 What are the symptoms of an allergic reaction?
14 How does diabetes affect the body?
15 What measures can be taken to prevent the spread of infectious diseases?
16 What are the different types of pain management techniques?
17 Can you describe the process of diagnosing a medical condition?
18 What are the potential complications of un

# Without lora model

In [ ]:
# Function to generate responses from the model
def generate_responses(model, question):
    prompt = f"Patient: \"{question}\"\nDoctor:"
    # Generate the answer from the model and then clean and extract the real model's response from the prompt engineered string
    output = clean_answer_text(model.generate(prompt, max_length=1024))
    return output

# Function clean the output of the model from the prompt engineering done in the "generate_responses" function
def clean_answer_text(text: str) -> str:
    # Define the start marker for the model's response
    doctor_response_start = text.find("Doctor:") + len("Doctor:")
    # Extract everything after "Doctor:"
    response_text = text[doctor_response_start:].strip()
    # If there's a follow-up "Patient:" in the response, cut the response there
    follow_up_index = response_text.find("\nPatient:")
    if follow_up_index != -1:
        response_text = response_text[:follow_up_index]
    # If there's no follow-up "Patient:", cut the response to the last dot (.)
    else:
        last_dot_index = response_text.rfind(".")
        if last_dot_index != -1:
            response_text = response_text[:last_dot_index + 1]
    # Additional cleaning if necessary (e.g., removing leading/trailing spaces or new lines)
    response_text = response_text.strip()
    response_text = response_text.replace("Doctor: ","")
    return response_text

In [ ]:
without_lora_model = keras.models.load_model("/content/drive/My Drive/SAE/Without_LoRA_Model_V1.keras")

/usr/local/lib/python3.10/dist-packages/keras/src/saving/serialization_lib.py:727: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 4 variables whereas the saved optimizer has 0 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 0 variables. 
  trackable.load_own_variables(weights_store.get(inner_path

In [ ]:
wb = Workbook()
wb = load_workbook(filename='/content/drive/My Drive/stat.xlsx')
sheet = wb['Feuil1']

#Without_LoRA_Model_V1.keras
i=2
for cell in sheet['A'][1:]:
    print(i-1, cell.value)
    sheet['C' + str(i) ] = generate_responses(without_lora_model, cell.value)
    wb.save('/content/drive/My Drive/stat.xlsx')
    i+=1
    if i > 31:
      break
wb.close()

1 What are the common causes of headaches?
2 How do over-the-counter pain relievers like ibuprofen work?
3 Can you explain the symptoms of a migraine?
4 What are the potential side effects of long-term medication use?
5 What are the differences between viral and bacterial infections?
6 Can you describe the stages of wound healing?
7 What is the significance of a healthy diet in preventing illness?
8 How does exercise affect mental health?
9 What are the warning signs of a heart attack?
10 How do vaccines work to prevent disease?
11 What is the role of antibiotics in treating bacterial infections?
12 Can you explain the concept of antibiotic resistance?
13 What are the symptoms of an allergic reaction?
14 How does diabetes affect the body?
15 What measures can be taken to prevent the spread of infectious diseases?
16 What are the different types of pain management techniques?
17 Can you describe the process of diagnosing a medical condition?
18 What are the potential complications of un

# LoRA_Model_V1

In [ ]:
# Function to generate responses from the model
def generate_responses(model, question):
    prompt = f"Patient: \"{question}\"\nDoctor:"
    # Generate the answer from the model and then clean and extract the real model's response from the prompt engineered string
    output = clean_answer_text(model.generate(prompt, max_length=1024))
    return output

# Function clean the output of the model from the prompt engineering done in the "generate_responses" function
def clean_answer_text(text: str) -> str:
    # Define the start marker for the model's response
    doctor_response_start = text.find("Doctor:") + len("Doctor:")
    # Extract everything after "Doctor:"
    response_text = text[doctor_response_start:].strip()
    # If there's a follow-up "Patient:" in the response, cut the response there
    follow_up_index = response_text.find("\nPatient:")
    if follow_up_index != -1:
        response_text = response_text[:follow_up_index]
    # If there's no follow-up "Patient:", cut the response to the last dot (.)
    else:
        last_dot_index = response_text.rfind(".")
        if last_dot_index != -1:
            response_text = response_text[:last_dot_index + 1]
    # Additional cleaning if necessary (e.g., removing leading/trailing spaces or new lines)
    response_text = response_text.strip()
    response_text = response_text.replace("Doctor: ","")
    return response_text

In [ ]:
model_v1 = keras.models.load_model("/content/drive/MyDrive/SAE/LoRA_Model_V1.keras")

/usr/local/lib/python3.10/dist-packages/keras/src/saving/serialization_lib.py:727: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 4 variables whereas the saved optimizer has 0 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 0 variables. 
  trackable.load_own_variables(weights_store.get(inner_path

In [ ]:
wb = Workbook()
wb = load_workbook(filename='/content/drive/MyDrive/SAE/stat.xlsx')
sheet = wb['Feuil1']

#LoRA_Model_V1.keras
i=2
for cell in sheet['A'][1:]:
    print(i-1, cell.value)
    sheet['D' + str(i) ] = generate_responses(model_v1, cell.value)
    wb.save('/content/drive/MyDrive/SAE/stat.xlsx')
    i+=1
    if i > 31:
      break
wb.close()


1 What are the common causes of headaches?
2 How do over-the-counter pain relievers like ibuprofen work?
3 Can you explain the symptoms of a migraine?
4 What are the potential side effects of long-term medication use?
5 What are the differences between viral and bacterial infections?
6 Can you describe the stages of wound healing?
7 What is the significance of a healthy diet in preventing illness?
8 How does exercise affect mental health?
9 What are the warning signs of a heart attack?
10 How do vaccines work to prevent disease?
11 What is the role of antibiotics in treating bacterial infections?
12 Can you explain the concept of antibiotic resistance?
13 What are the symptoms of an allergic reaction?
14 How does diabetes affect the body?
15 What measures can be taken to prevent the spread of infectious diseases?
16 What are the different types of pain management techniques?
17 Can you describe the process of diagnosing a medical condition?
18 What are the potential complications of un

# LoRA_Model_V2

In [ ]:
# Function to generate responses from the model
def generate_responses(model, question):
    prompt = f"Patient: \"{question}\"\nDoctor:"
    # Generate the answer from the model and then clean and extract the real model's response from the prompt engineered string
    output = clean_answer_text(model.generate(prompt, max_length=1024))
    return output

# Function clean the output of the model from the prompt engineering done in the "generate_responses" function
def clean_answer_text(text: str) -> str:
    # Define the start marker for the model's response
    doctor_response_start = text.find("Doctor:") + len("Doctor:")
    # Extract everything after "Doctor:"
    response_text = text[doctor_response_start:].strip()
    # If there's a follow-up "Patient:" in the response, cut the response there
    follow_up_index = response_text.find("\nPatient:")
    if follow_up_index != -1:
        response_text = response_text[:follow_up_index]
    # If there's no follow-up "Patient:", cut the response to the last dot (.)
    else:
        last_dot_index = response_text.rfind(".")
        if last_dot_index != -1:
            response_text = response_text[:last_dot_index + 1]
    # Additional cleaning if necessary (e.g., removing leading/trailing spaces or new lines)
    response_text = response_text.strip()
    response_text = response_text.replace("Doctor: ","")
    return response_text

In [ ]:
model_v2 = keras.models.load_model("/content/drive/MyDrive/SAE/LoRA_Model_V2.keras")

In [ ]:
wb = Workbook()
wb = load_workbook(filename='/content/drive/MyDrive/SAE/stat.xlsx')
sheet = wb['Feuil1']

#LoRA_Model_V2.keras
i=2
for cell in sheet['A'][1:]:
    print(i-1, cell.value)
    sheet['E' + str(i) ] = generate_responses(model_v2, cell.value)
    wb.save('/content/drive/MyDrive/SAE/stat.xlsx')
    i+=1
    if i > 31:
      break
wb.close()

1 What are the common causes of headaches?
2 How do over-the-counter pain relievers like ibuprofen work?
3 Can you explain the symptoms of a migraine?
4 What are the potential side effects of long-term medication use?
5 What are the differences between viral and bacterial infections?
6 Can you describe the stages of wound healing?
7 What is the significance of a healthy diet in preventing illness?
8 How does exercise affect mental health?
9 What are the warning signs of a heart attack?
10 How do vaccines work to prevent disease?
11 What is the role of antibiotics in treating bacterial infections?
12 Can you explain the concept of antibiotic resistance?
13 What are the symptoms of an allergic reaction?
14 How does diabetes affect the body?
15 What measures can be taken to prevent the spread of infectious diseases?
16 What are the different types of pain management techniques?
17 Can you describe the process of diagnosing a medical condition?
18 What are the potential complications of un

# LoRA_Model_CP3

In [ ]:
def clean_answer_text(text: str) -> str:
    # Define the start marker for the model's response
    response_start = text.find("[ANSWER]") + len("[ANSWER]")

    # Extract everything after "Doctor:"
    response_text = text[response_start:].strip()
    last_dot_index = response_text.rfind(".")
    if last_dot_index != -1:
      response_text = response_text[:last_dot_index + 1]

    # Additional cleaning if necessary (e.g., removing leading/trailing spaces or new lines)
    response_text = response_text.strip()

    return response_text


def generate_responses(model, question):
    prompt = f"[QUESTION] {question} [ANSWER]"
    output = model.generate(prompt, max_length=1024)
    # Clean and extract the model's response from `output`
    return clean_answer_text(output)

In [ ]:
model_loraCP3 = keras.models.load_model("/content/drive/MyDrive/LoRA_Model_CP3.keras")

/usr/local/lib/python3.10/dist-packages/keras/src/saving/serialization_lib.py:727: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 4 variables whereas the saved optimizer has 0 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 0 variables. 
  trackable.load_own_variables(weights_store.get(inner_path

In [ ]:
wb = Workbook()
wb = load_workbook(filename='/content/drive/MyDrive/stat.xlsx')
sheet = wb['Feuil1']


i=2
for cell in sheet['A'][1:]:
    print(i-1, cell.value)
    sheet['H' + str(i) ] = generate_responses(model_loraCP3, cell.value)
    wb.save('/content/drive/MyDrive/stat.xlsx')
    i+=1
    if i > 31:
      break
wb.close()

1 What are the common causes of headaches?
2 How do over-the-counter pain relievers like ibuprofen work?
3 Can you explain the symptoms of a migraine?
4 What are the potential side effects of long-term medication use?
5 What are the differences between viral and bacterial infections?
6 Can you describe the stages of wound healing?
7 What is the significance of a healthy diet in preventing illness?
8 How does exercise affect mental health?
9 What are the warning signs of a heart attack?
10 How do vaccines work to prevent disease?
11 What is the role of antibiotics in treating bacterial infections?
12 Can you explain the concept of antibiotic resistance?
13 What are the symptoms of an allergic reaction?
14 How does diabetes affect the body?
15 What measures can be taken to prevent the spread of infectious diseases?
16 What are the different types of pain management techniques?
17 Can you describe the process of diagnosing a medical condition?
18 What are the potential complications of un